# 🎯 RIQ MACHINE - NBA Prediction & Analysis

## Features:
- ✅ Load trained neural hybrid models (TabNet + LightGBM)
- ✅ Fetch live NBA games and player stats
- ✅ Generate predictions with full feature engineering
- ✅ Compare to betting lines (find value)
- ✅ Backtest on historical data
- ✅ Track performance and ROI
- 🆕 Safe Mode: Conservative betting with extra margin buffer

## Prerequisites:
1. Trained models from `NBA_COLAB_SIMPLE.ipynb`
2. Three separate files to upload:
   - `nba_models_trained.zip` (trained models)
   - `priors_data.zip` (Basketball Reference priors)
   - `PlayerStatistics.csv` (historical data for backtesting)

## Quick Start:
1. Run Setup cell
2. Upload 3 files (separate prompts)
3. Configure API keys
4. **Optional**: Enable Safe Mode for conservative picks
5. Run analysis to get predictions

---

## 📦 Setup & Installation

In [ ]:
# ============================================================
# SETUP - Install packages and download LATEST code
# ============================================================

print("="*70)
print("📦 SETUP - Installing packages and pulling latest code")
print("="*70)

print("\n1️⃣ Installing packages...")
!pip install -q nba-api pytorch-tabnet lightgbm scikit-learn pandas numpy requests
print("   ✅ Packages installed")

print("\n2️⃣ Cleaning up old code...")
import os
import shutil

os.chdir('/content')

# FORCE remove old code to get latest updates
if os.path.exists('meep'):
    shutil.rmtree('meep')
    print("   🧹 Removed cached code")

print("\n3️⃣ Cloning latest code from GitHub...")
!git clone -q https://github.com/tyriqmiles0529-pixel/meep.git
os.chdir('meep')
print("   ✅ Code downloaded")

print("\n4️⃣ Verifying code version...")
!git log --oneline -n 3
print("\n   ⚠️  VERIFY: You should see latest commits including:")
print("   ⚠️  - Safe mode status banner")
print("   ⚠️  - Evaluation cells rebuild")
print("   ⚠️  If not, re-run this cell to pull latest updates")

# Add to Python path
import sys
sys.path.insert(0, '/content/meep')

print("\n" + "="*70)
print("✅ SETUP COMPLETE!")
print("="*70)
print("\nNext steps:")
print("  1. Run 'Upload Models & Data' cell")
print("  2. Run 'Configure API Keys' cell")
print("  3. (Optional) Run 'Safe Mode Configuration' cell")
print("  4. Run 'Full Analysis' cell")
print("\n⚠️  IMPORTANT: If you see model warnings about 56 vs 95 features,")
print("   re-run THIS cell to pull the latest riq_analyzer.py updates!")

---

## 📤 Upload Models & Data

In [ ]:
# ============================================================
# UPLOAD - Models, priors data, and player statistics
# ============================================================

from google.colab import files
import os
import zipfile

os.chdir('/content')

print("="*70)
print("📤 UPLOAD ALL FILES")
print("="*70)
print("\nUpload ALL 3 files at once:")
print("  1. nba_models_trained.zip (your trained models)")
print("  2. priors_data.zip (Basketball Reference priors)")
print("  3. PlayerStatistics.csv.zip (historical data)")
print("\nUploading...\n")

uploaded = files.upload()

# Extract models
if os.path.exists('nba_models_trained.zip'):
    print("\n📦 Extracting models...")
    !unzip -q nba_models_trained.zip
    !rm nba_models_trained.zip
    print("✅ Models extracted to ./models/ and ./model_cache/")
else:
    print("⚠️  nba_models_trained.zip not found")

# Extract priors
if os.path.exists('priors_data.zip'):
    print("\n📦 Extracting priors...")
    !unzip -q priors_data.zip
    !rm priors_data.zip
    print("✅ Priors extracted to ./priors_data/")
else:
    print("⚠️  priors_data.zip not found")

# Extract player stats
if os.path.exists('PlayerStatistics.csv.zip'):
    print("\n📦 Extracting PlayerStatistics.csv...")
    !unzip -q PlayerStatistics.csv.zip
    !rm PlayerStatistics.csv.zip
    if os.path.exists('PlayerStatistics.csv'):
        size_mb = os.path.getsize('PlayerStatistics.csv') / 1024 / 1024
        print(f"✅ PlayerStatistics.csv extracted ({size_mb:.1f} MB)")
    else:
        print("❌ Extraction failed")
else:
    print("⚠️  PlayerStatistics.csv.zip not found")

# Final verification
print("\n" + "="*70)
print("🔍 FINAL VERIFICATION")
print("="*70)

if os.path.exists('models'):
    models = [f for f in os.listdir('models') if f.endswith('.pkl')]
    print(f"✅ Found {len(models)} model files")
else:
    print("❌ models/ directory not found")

if os.path.exists('priors_data'):
    priors = [f for f in os.listdir('priors_data') if f.endswith('.csv')]
    print(f"✅ Found {len(priors)} priors files")
else:
    print("❌ priors_data/ directory not found")

if os.path.exists('PlayerStatistics.csv'):
    size_mb = os.path.getsize('PlayerStatistics.csv') / 1024 / 1024
    print(f"✅ PlayerStatistics.csv ready ({size_mb:.1f} MB)")
else:
    print("❌ PlayerStatistics.csv not found")

print("\n✅ Upload complete!")
print("\nNext: Configure API keys and run analysis")

---

## 🔑 Configure API Keys

In [ ]:
# ============================================================
# API KEYS - Required for betting odds
# ============================================================

import os

# Your API keys (from keys.py)
API_SPORTS_KEY = "4979ac5e1f7ae10b1d6b58f1bba01140"
RAPIDAPI_KEY = "9ef7289093msh76adf5ee5bedb5fp15e0d6jsnc2a0d0ed9abe"
THEODDS_API_KEY = "c98703301e8f89ef2c3648a4373939fd"

# Set environment variables
os.environ["API_SPORTS_KEY"] = API_SPORTS_KEY
os.environ["APISPORTS_KEY"] = API_SPORTS_KEY
os.environ["RAPIDAPI_KEY"] = RAPIDAPI_KEY
os.environ["THEODDS_API_KEY"] = THEODDS_API_KEY

print("="*70)
print("🔑 API KEY STATUS")
print("="*70)
print(f"API-Sports: ✅ Set ({API_SPORTS_KEY[:8]}...)")
print(f"RapidAPI:   ✅ Set ({RAPIDAPI_KEY[:8]}...)")
print(f"The Odds:   ✅ Set ({THEODDS_API_KEY[:8]}...)")
print("\n✅ All API keys configured!")
print("\nNext: Run analysis cell to fetch games and generate predictions")

---

## 🛡️ Safe Mode Configuration (Optional)

In [ ]:
# ============================================================
# SAFE MODE - Conservative betting with extra margin
# ============================================================

import os

# SAFE MODE SETTINGS
# Set to True for conservative betting (adds extra buffer to lines)
SAFE_MODE = True  # Change to False for standard analysis
SAFE_MARGIN = 1.0  # Extra points/rebounds/assists buffer (0.5-1.5 recommended)

# Set environment variables
if SAFE_MODE:
    os.environ["SAFE_MODE"] = "true"
    os.environ["SAFE_MARGIN"] = str(SAFE_MARGIN)
    print("="*70)
    print("🛡️  SAFE MODE: ON")
    print("="*70)
    print(f"Safety Margin: {SAFE_MARGIN} points/rebounds/assists\n")
    print("How it works:")
    print("  • UNDER bets: Requires line at ORIGINAL + MARGIN")
    print("  • OVER bets: Requires line at ORIGINAL - MARGIN\n")
    print("Example:")
    print("  Projection: 2.8 assists")
    print("  Original Line: 3.5 assists")
    print(f"  Safe Mode Line: 3.5 + {SAFE_MARGIN} = {3.5 + SAFE_MARGIN} assists")
    print("  Pick: UNDER 4.5 (only if you can find this line)\n")
    print("Benefits:")
    print("  ✅ More conservative picks")
    print("  ✅ Extra safety buffer")
    print("  ✅ 30-50% fewer picks, but higher confidence\n")
    print("⚠️  Note: You must find the safer line at your sportsbook!")
else:
    # Unset environment variables
    if "SAFE_MODE" in os.environ:
        del os.environ["SAFE_MODE"]
    if "SAFE_MARGIN" in os.environ:
        del os.environ["SAFE_MARGIN"]
    print("="*70)
    print("⚡ SAFE MODE: OFF")
    print("="*70)
    print("Standard analysis mode - maximum number of picks\n")

print("✅ Safe mode configured!")
print("\nNext: Run analysis to see picks")

---

## 🎯 Run Full Analysis

In [ ]:
# ============================================================
# RUN FULL ANALYSIS - Uses riq_analyzer.py
# ============================================================

import sys
sys.path.insert(0, '/content/meep')

# Import and run riq_analyzer
print("="*72)
print("🎯 RUNNING FULL NBA PREDICTION ANALYSIS")
print("="*72)
print("\nThis uses riq_analyzer.py to:")
print("  1. Fetch upcoming NBA games (API-Sports)")
print("  2. Fetch betting odds (TheRundown/RapidAPI)")
print("  3. Load trained models")
print("  4. Generate predictions")
print("  5. Compare predictions vs lines")
print("  6. Calculate EV, Kelly, ELG scores")
print("  7. Build optimal parlays")
print("\n" + "="*72 + "\n")

try:
    # Import riq_analyzer and run analysis
    import riq_analyzer
    
    # Run the analysis function
    riq_analyzer.run_analysis()
    
except ImportError as e:
    print(f"❌ Failed to import riq_analyzer: {e}")
    print("\n⚠️  Make sure you ran the Setup cell to clone the repo")
    
except Exception as e:
    print(f"❌ Analysis failed: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Possible issues:")
    print("  - API keys not set (run API Keys cell)")
    print("  - No games today")
    print("  - API rate limits exceeded")
    print("  - Models not uploaded")

---

## 📥 Download Results (IMPORTANT!)

In [ ]:
# ============================================================
# DOWNLOAD RESULTS - Get predictions for local backup
# ============================================================
# CRITICAL: Run this after analysis to save predictions!
# ============================================================

from google.colab import files
import os
import glob

print("="*70)
print("📥 DOWNLOADING PREDICTION FILES")
print("="*70)

# Search current directory first (where analysis saves files)
print("\n🔍 Searching for files...\n")

# Method 1: Try current directory
ledger_found = False
analysis_found = False

if os.path.exists("bets_ledger.pkl"):
    print("1️⃣ Downloading bets_ledger.pkl...")
    files.download("bets_ledger.pkl")
    print("   ✅ Downloaded: bets_ledger.pkl\n")
    ledger_found = True
else:
    # Method 2: Search recursively
    ledgers = glob.glob("**/bets_ledger.pkl", recursive=True)
    if ledgers:
        print(f"1️⃣ Downloading {ledgers[0]}...")
        files.download(ledgers[0])
        print(f"   ✅ Downloaded: {ledgers[0]}\n")
        ledger_found = True

if not ledger_found:
    print("❌ No bets_ledger.pkl found\n")

# Find latest analysis file
analysis_files = sorted(glob.glob("prop_analysis_*.json"))
if not analysis_files:
    analysis_files = sorted(glob.glob("**/prop_analysis_*.json", recursive=True))

if analysis_files:
    latest = analysis_files[-1]
    print(f"2️⃣ Downloading {os.path.basename(latest)}...")
    files.download(latest)
    print(f"   ✅ Downloaded: {os.path.basename(latest)}\n")
    analysis_found = True
else:
    print("❌ No prop_analysis files found\n")

print("="*70)
if ledger_found or analysis_found:
    print("✅ DOWNLOAD COMPLETE!")
    print("="*70)
    print("\nWhat to do with these files:")
    print("  1. Save them to your computer")
    print("  2. Copy bets_ledger.pkl to C:\\Users\\tmiles11\\nba_predictor\\")
    print("  3. Upload bets_ledger.pkl to Evaluate_Predictions.ipynb")
    print("     to settle bets and calibrate models!")
else:
    print("⚠️ NO FILES DOWNLOADED")
    print("="*70)
    print("\nDid the analysis complete successfully?")
    print("Scroll up and check for:")
    print("  • Results saved to: prop_analysis_...")
    print("  • ✅ X props meet ELG gates")
    print("\nIf you see those, files should exist in current directory.")
    print("\nCurrent directory contents:")
    for item in sorted(os.listdir(".")):
        if item.endswith(".pkl") or item.endswith(".json"):
            print(f"  📄 {item}")

print("\n" + "="*70)
